# **ROS** Protein homo

## Required libraries and configuration

In [1]:
!pip install -q flwr[simulation]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 31.1 MB/s eta 0:00:00



Import required libraries

In [2]:
import os

import re
import time
import random
import sys

from typing import Dict, Optional, Tuple, List
from collections import OrderedDict

import flwr as fl
from flwr.common import Metrics

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow.keras import models, layers, losses, metrics, optimizers

from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import SMOTE, RandomOverSampler

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics as skmetrics
from sklearn.metrics import confusion_matrix

Define some parameters for the simulation, such as the number of clients in the federated scenario, the number of federated rounds, the number of epochs of each client before communicating, and the batch size for training phase

In [4]:
# Some parameters
NUM_CLIENTS = 10 # Number of clients in the federated scenario
NUM_ROUNDS = 10 #5 Number of learning rounds in the federated computation
NUM_EPOCHS = 5 #3 Number of epochs that the local dataset is seen each round
BATCH_SIZE = 20 # Batch size for training phase

# Define the seed for random numbers
seed = 10
np.random.seed(seed)
tf.random.set_seed(seed)
tf.keras.utils.set_random_seed(seed)
random.seed(seed)

## Loading and preparing the input data

The Sentiment140 dataset is not available in TFF, so it needs to be loaded from other source such as the tfds (tensorflow datasets) library. Then, it is adapted to the TFF format, so it can be used to train a model using TFF.

Note: We download the full Sentiment140 dataset, but in order to be able to execute the experiments in reasonable time, we are going to use a portion of it. In this notebook we are selecting just 1% of the data for training and 10% for testing purposes.

In [5]:
dset_name = 'protein_homo'
dset = fetch_datasets()[dset_name]

In [6]:
X = pd.DataFrame(StandardScaler().fit_transform(dset.data))
y = dset.target

y[y == -1] = 0
X['label'] = y

dtrain, dtest = train_test_split(X, test_size=0.2, random_state=42, stratify=y)

In [7]:
dtest = dtest.iloc[:-1,:]

In [8]:
dtrain.shape

(116600, 75)

In [9]:
dtrain['label'].value_counts()

0    115563
1      1037
Name: label, dtype: int64

Select the text and polarity columns from the original data, and transform it to a dataframe so it can be later used in TFF.

At this point, we do not select the 'user' column because we will create random IID partitions, so the 'user' column is not neccesary. If the user want to try it with non-IID partition, he/she may also keep the 'user' column and use it as client identifier instead of the following random user ID.

In [10]:
# Create a random list of ids. Each instance is given a random id, indicating the client where will be distributed
ids_train = [i for i in range(NUM_CLIENTS) for _ in range(len(dtrain)//NUM_CLIENTS)]
random.Random(seed).shuffle(ids_train)

# Add the id assignment to the dataframe
dtrain['user'] = ids_train
unique_ids_train = list(set(ids_train))

In [11]:
# Do the same with the test data
ids_test = [i for i in range(NUM_CLIENTS) for _ in range(len(dtest)//NUM_CLIENTS)]
random.Random(seed+1).shuffle(ids_test)
dtest['user'] = ids_test
unique_ids_test = list(set(ids_test))

For the sake of simplicity, in this notebook we will be dealing with a binary problem. For that purpose, we remove the neutral tweets, so the classifier's aim is to differentiate between positive and negative tweets.

Create the DataLoaders for each client. It creates an array of datasets, one for each client.

In [12]:
# Create DataLoaders for each client
train_data = []
test_data = []
for i in range(NUM_CLIENTS):
    train_data.append(dtrain.loc[dtrain['user']==unique_ids_train[i]])
    test_data.append(dtest.loc[dtest['user']==unique_ids_test[i]])

## Create a Deep Learning model

In this case we use a model composed by a pre-trained model from tfhub, as well as dense layers. The pre-trained model is not updated in the example; however, the ``trainable`` parameter can be set to ``True``, so such layers are also fine-tuned in the collaborative training.

Note that any network architecture supported by keras can be used. Besides, Flower also supports PyTorch models, but in contrast to Use Case 1, we use a TFF model here to provide a variety of options.

In [13]:
def create_keras_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(256, activation='relu', input_shape=[74]))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(1))

    # Compile the model
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.optimizers.Adam(),
        metrics=[tf.metrics.AUC(from_logits=True)]
    )

    return model

## Training in the federated scenario

First, we create a FlowerClient class, that includes the information of each simulated client. The class has three methods:
 * `get_parameters`: Get the parameters of the model to send them to the server
 * `fit`: Reveives the model parameters from the server, trains it with local data, and return the updated model parameters to the server
 * `evaluate`: Receives the model from the server and evaluates it with local data

In [14]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res

    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1, random_state=random_state)

    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [15]:
# Class-independent methods to get and set the parameters of a model
def get_parameters(model):
        return model.get_weights()

def set_parameters(model, parameters):
    model.set_weights(parameters)

In [16]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, client_train_data, client_test_data):
        # Init the client setting the x (text) and the y (polarity) for both train and testing
        self.model = model

        client_train_data = client_train_data.drop('user', axis=1)
        client_test_data = client_test_data.drop('user', axis=1)

        x_train, y_train = client_train_data.drop('label',axis=1), client_train_data['label'].to_numpy()
        x_test, y_test = client_test_data.drop('label',axis=1), client_test_data['label'].to_numpy()

        # ROS
        ros = RandomOverSampler(random_state=42, sampling_strategy=.7) # N_rm / N_M
        X_res, y_res = ros.fit_resample(x_train, y_train)
        # RUS
        X_eq, y_eq = RUS(X_res, y_res, frac=.05)

        self.x_train, self.y_train = X_eq, y_eq
        self.x_test, self.y_test = x_test, y_test

    def get_properties(self, config):
        """Get properties of client."""
        raise Exception("Not implemented")

    def get_parameters(self, config):
        """Get parameters of the local model."""
        return get_parameters(self.model)

    def fit(self, parameters, config):
        """Train parameters on the locally held training set."""
        # Update local model parameters
        set_parameters(self.model, parameters)

        # Get hyperparameters for this round
        batch_size: int = BATCH_SIZE
        epochs: int = NUM_EPOCHS

        # Train the model using hyperparameters from config
        history = self.model.fit(
            self.x_train,
            self.y_train,
            batch_size,
            epochs,
            verbose=0
        )

        # Return updated model parameters and results
        parameters_prime = get_parameters(model=self.model)
        num_examples_train = len(self.x_train)
        results = {
            "loss": history.history["loss"][0],
            #"accuracy": history.history["accuracy"][0],

        }
        return parameters_prime, num_examples_train, results

    def evaluate(self, parameters, config):
        """Evaluate parameters on the locally held test set."""

        # Update local model with global parameters
        set_parameters(self.model, parameters)

        # Get config values
        steps: int = 10

        # Evaluate global model parameters on the local test data and return results
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, 1)
        num_examples_test = len(self.x_test)

        y_pred = self.model.predict(self.x_test)
        y_pred = [1 if pred >= 0.5 else 0 for pred in y_pred]

        cm = confusion_matrix(self.y_test, y_pred)
        tn = cm[0][0]
        fn = cm[1][0]
        tp = cm[1][1]
        fp = cm[0][1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)

        cr = skmetrics.classification_report(self.y_test, y_pred)
        print(cr)

        return loss, num_examples_test, {"TPR*TNR": tpr*tnr}

To simulate the federated scenario in a single machine, the client_fn method allows to create FlowerClients on demand, given the client id.

Note that each client is passed both training and testing local data, so the evaluation over test data is done during the simulation itself.

In [17]:
def client_fn(cid: str) -> FlowerClient:
    # Load model
    model = create_keras_model()

    # Note: each client gets a different train/test data
    client_train_data = train_data[int(cid)]
    client_test_data = test_data[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(model, client_train_data, client_test_data)

In order to show averaged evaluations metrics beyond loss, we should define a method to do that; in this case, the accuracy is weighted averaged.

In [18]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["TPR*TNR"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"TPR*TNR": sum(accuracies) / sum(examples)}

Define a training strategy with the weighted FedAvg algorithm.

Then, start the simulation indicating the method to create clients, the number of clients in the simulation, the number of rounds, and the strategy (i.e., the FedAvg strategy to combine local updates). The simulation covers both the federated model training as well as evaluating the model with each local test data.

In [19]:
# Define the configuration during training/fitting the model. It is used later when defining the FedAvg strategy.
def fit_config(server_round: int):
    config = {
        "batch_size": BATCH_SIZE,
        "local_epochs": NUM_EPOCHS
    }
    return config

# Define the configuration during evaluation. It is used later when defining the FedAvg strategy.
def evaluate_config(server_round: int):
    return {"val_steps": 5}

In [20]:
# Define strategy
strategy = fl.server.strategy.FedAvg(
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    evaluate_metrics_aggregation_fn=weighted_average,
)

# Start simulation
fl_sim = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
)

INFO flwr 2023-07-20 12:56:55,571 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-07-20 12:56:58,415	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-07-20 12:57:00,462 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7890287003.0, 'object_store_memory': 3945143500.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7890287003.0, 'object_store_memory': 3945143500.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-07-20 12:57:00,472 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-07-20 12:57:00,475 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=1923) 2023-07-20 12:57:02.633220: W tensorflow/compiler

79/92 [========================>.....] - ETA: 0s
(launch_and_evaluate pid=1923)               precision    recall  f1-score   support
(launch_and_evaluate pid=1923) 
(launch_and_evaluate pid=1923)            0       1.00      0.99      0.99      2892
(launch_and_evaluate pid=1923)            1       0.31      0.83      0.45        23
(launch_and_evaluate pid=1923) 
(launch_and_evaluate pid=1923)     accuracy                           0.98      2915
(launch_and_evaluate pid=1923)    macro avg       0.65      0.91      0.72      2915
(launch_and_evaluate pid=1923) weighted avg       0.99      0.98      0.99      2915
(launch_and_evaluate pid=1923) 
1773/2915 [=================>............] - ETA: 2s - loss: 0.0294 - auc_7: 0.9989
(launch_and_evaluate pid=1924)  [repeated 5x across cluster]
83/92 [==========================>...] - ETA: 0s
(launch_and_evaluate pid=1924)               precision    recall  f1-score   support
(launch_and_evaluate pid=1924) 
(launch_and_evaluate pid=1924)    

DEBUG flwr 2023-07-20 12:59:07,071 | server.py:182 | evaluate_round 1 received 10 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 10 results and 0 failures
DEBUG flwr 2023-07-20 12:59:07,074 | server.py:218 | fit_round 2: strategy sampled 10 clients (out of 10)
DEBUG:flwr:fit_round 2: strategy sampled 10 clients (out of 10)


92/92 [==============================] - 1s 4ms/step
(launch_and_evaluate pid=1923)               precision    recall  f1-score   support
(launch_and_evaluate pid=1923) 
(launch_and_evaluate pid=1923)            0       1.00      0.99      0.99      2899
(launch_and_evaluate pid=1923)            1       0.28      1.00      0.44        16
(launch_and_evaluate pid=1923) 
(launch_and_evaluate pid=1923)     accuracy                           0.99      2915
(launch_and_evaluate pid=1923)    macro avg       0.64      0.99      0.72      2915
(launch_and_evaluate pid=1923) weighted avg       1.00      0.99      0.99      2915
(launch_and_evaluate pid=1923) 


DEBUG flwr 2023-07-20 13:00:10,554 | server.py:232 | fit_round 2 received 10 results and 0 failures
DEBUG:flwr:fit_round 2 received 10 results and 0 failures
DEBUG flwr 2023-07-20 13:00:10,584 | server.py:168 | evaluate_round 2: strategy sampled 10 clients (out of 10)
DEBUG:flwr:evaluate_round 2: strategy sampled 10 clients (out of 10)


...

2503/2915 [========================>.....] - ETA: 1s - loss: 0.0708 - auc_89: 0.9705 [repeated 4x across cluster]


DEBUG flwr 2023-07-20 13:16:14,910 | server.py:232 | fit_round 10 received 10 results and 0 failures
DEBUG:flwr:fit_round 10 received 10 results and 0 failures
DEBUG flwr 2023-07-20 13:16:14,955 | server.py:168 | evaluate_round 10: strategy sampled 10 clients (out of 10)
DEBUG:flwr:evaluate_round 10: strategy sampled 10 clients (out of 10)


   1/2915 [..............................] - ETA: 33:12 - loss: 1.0414e-11 - auc_96: 0.0000e+00
(launch_and_evaluate pid=1924)  [repeated 5x across cluster]
92/92 [==============================] - 0s 2ms/step
(launch_and_evaluate pid=1924)               precision    recall  f1-score   support
(launch_and_evaluate pid=1924) 
(launch_and_evaluate pid=1924)            0       1.00      0.99      0.99      2887
(launch_and_evaluate pid=1924)            1       0.47      0.82      0.60        28
(launch_and_evaluate pid=1924) 
(launch_and_evaluate pid=1924)     accuracy                           0.99      2915
(launch_and_evaluate pid=1924)    macro avg       0.73      0.91      0.80      2915
(launch_and_evaluate pid=1924) weighted avg       0.99      0.99      0.99      2915
(launch_and_evaluate pid=1924) 
(launch_and_evaluate pid=1924) 
1243/2915 [===========>..................] - ETA: 7s - loss: 0.0565 - auc_96: 0.9250 [repeated 2x across cluster]
(launch_and_evaluate pid=1923) 
1130/2

DEBUG flwr 2023-07-20 13:17:14,656 | server.py:182 | evaluate_round 10 received 10 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 10 results and 0 failures
INFO flwr 2023-07-20 13:17:14,659 | server.py:147 | FL finished in 1209.280548952
INFO:flwr:FL finished in 1209.280548952
INFO flwr 2023-07-20 13:17:14,662 | app.py:218 | app_fit: losses_distributed [(1, 0.05221325382590294), (2, 0.05652711279690266), (3, 0.05689622014760971), (4, 0.058424098789691924), (5, 0.058049104362726214), (6, 0.06408430114388466), (7, 0.05593122653663159), (8, 0.06081847213208676), (9, 0.0638232983648777), (10, 0.059821445122361185)]
INFO:flwr:app_fit: losses_distributed [(1, 0.05221325382590294), (2, 0.05652711279690266), (3, 0.05689622014760971), (4, 0.058424098789691924), (5, 0.058049104362726214), (6, 0.06408430114388466), (7, 0.05593122653663159), (8, 0.06081847213208676), (9, 0.0638232983648777), (10, 0.059821445122361185)]
INFO flwr 2023-07-20 13:17:14,665 | app.py:219 | app_fit: metrics

92/92 [==============================] - 0s 2ms/step


## Evaluation with test data

The evaluation has been done during the simulation. Following, we show the averaged results over test data.
The result of the simulation includes the results on all rounds, so we retrieve those of the last round.

In [21]:
print('Test data, \t Loss={:.4f}, \t Accuracy={:.4f}'.format(fl_sim.losses_distributed[-1][1], fl_sim.metrics_distributed['TPR*TNR'][-1][1]))

Test data, 	 Loss=0.0598, 	 Accuracy=0.9018


In [22]:
fl_sim

History (loss, distributed):
	round 1: 0.05221325382590294
	round 2: 0.05652711279690266
	round 3: 0.05689622014760971
	round 4: 0.058424098789691924
	round 5: 0.058049104362726214
	round 6: 0.06408430114388466
	round 7: 0.05593122653663159
	round 8: 0.06081847213208676
	round 9: 0.0638232983648777
	round 10: 0.059821445122361185
History (metrics, distributed, evaluate):
{'TPR*TNR': [(1, 0.8801737814687041), (2, 0.8879617552039559), (3, 0.89188937130901), (4, 0.8883952092883255), (5, 0.8924153844522027), (6, 0.8964603894732699), (7, 0.8970013039721367), (8, 0.9013374001765005), (9, 0.9079704324772048), (10, 0.9017968078673672)]}